In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import sys

sys.path.append("../")

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import DPTForDepthEstimation

os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

from src.config import PROCESSED_SALICON_PATH
from src.datasets.salicon_dataset import SaliconDataset
from src.utils.file import get_paths_recursive

In [23]:
from transformers import AutoModelForDepthEstimation 

model_name = "depth-anything/Depth-Anything-V2-Base-hf"
base_model = AutoModelForDepthEstimation.from_pretrained(model_name, output_hidden_states=True)
base_model.eval()
base_model.to("cuda")
input = torch.randn(1, 3, 224, 224).to("cuda")
output = base_model(input)
hidden_states = output.hidden_states
feature_maps = []
for hidden_state in hidden_states:
    patch_embeddings = hidden_state[:, 1:, :]
    size = int(np.sqrt(patch_embeddings.shape[1]))
    feature_map = patch_embeddings.reshape(1, size, size, -1).permute(0, 3, 1, 2)
    feature_maps.append(feature_map)
    print(feature_map.shape)

torch.Size([1, 768, 16, 16])
torch.Size([1, 768, 16, 16])
torch.Size([1, 768, 16, 16])
torch.Size([1, 768, 16, 16])
torch.Size([1, 768, 16, 16])
torch.Size([1, 768, 16, 16])
torch.Size([1, 768, 16, 16])
torch.Size([1, 768, 16, 16])
torch.Size([1, 768, 16, 16])
torch.Size([1, 768, 16, 16])
torch.Size([1, 768, 16, 16])
torch.Size([1, 768, 16, 16])
torch.Size([1, 768, 16, 16])


In [29]:
output

DepthEstimatorOutput(loss=None, predicted_depth=tensor([[[3.6459, 3.6371, 3.6382,  ..., 3.8307, 3.8642, 3.4359],
         [3.5851, 3.5965, 3.6461,  ..., 3.8302, 3.8002, 3.8375],
         [3.6379, 3.6354, 3.6346,  ..., 3.8314, 3.8384, 3.8129],
         ...,
         [3.7470, 3.7505, 3.7726,  ..., 3.4320, 3.4412, 3.4475],
         [3.7737, 3.7316, 3.7884,  ..., 3.4462, 3.4444, 3.4539],
         [3.8146, 3.7688, 3.7746,  ..., 3.4340, 3.4360, 3.4613]]],
       device='cuda:0', grad_fn=<SqueezeBackward1>), hidden_states=(tensor([[[-0.0021, -0.0014, -0.0921,  ...,  0.0141,  0.0094,  0.0022],
         [-0.4140,  0.0083, -0.1673,  ...,  0.0166,  0.1997, -0.2020],
         [-0.0549, -0.0288, -0.1283,  ...,  0.2133, -0.2188,  0.1699],
         ...,
         [ 0.2361,  0.0428, -0.0536,  ..., -0.0881,  0.1829, -0.1304],
         [ 0.2749, -0.0476,  0.0102,  ...,  0.3193, -0.1028,  0.3406],
         [-0.0956,  0.0676,  0.0994,  ...,  0.0054,  0.0290,  0.3170]]],
       device='cuda:0', grad_fn=<Add

In [19]:
print(output.keys())
print(output["predicted_depth"].shape)
print([x.shape for x in output["hidden_states"]])

odict_keys(['predicted_depth', 'hidden_states'])
torch.Size([1, 224, 224])
[torch.Size([1, 257, 768]), torch.Size([1, 257, 768]), torch.Size([1, 257, 768]), torch.Size([1, 257, 768]), torch.Size([1, 257, 768]), torch.Size([1, 257, 768]), torch.Size([1, 257, 768]), torch.Size([1, 257, 768]), torch.Size([1, 257, 768]), torch.Size([1, 257, 768]), torch.Size([1, 257, 768]), torch.Size([1, 257, 768]), torch.Size([1, 257, 768])]


In [ ]:
from transformers import AutoModelForDepthEstimation 
import torch.nn as nn
import torch  

class DepthModelWithFeatures(nn.Module):     
    def __init__(self, model_name):         
        super().__init__()         
        self.base_model = AutoModelForDepthEstimation.from_pretrained(model_name)      

    def forward(self, **kwargs):         
        encoder_outputs = self.base_model(**kwargs)          
        hidden_features = encoder_outputs.hidden_states  
        predicted_depth = self.base_model(**kwargs).predicted_depth  

        return hidden_features, predicted_depth 
        
model_name = "depth-anything/Depth-Anything-V2-Base-hf"
model = DepthModelWithFeatures(model_name=model_name)
input = torch.rand(1, 3, 224, 224)
hidden_features, predicted_depth = model(input)
print([x.shape for x in hidden_features])
print(predicted_depth.shape)

TypeError: DepthAnythingForDepthEstimation.forward() got an unexpected keyword argument 'input'

In [3]:
#model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
#model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)
midas = torch.hub.load("intel-isl/MiDaS", model_type)

Using cache found in C:\Users\arnau/.cache\torch\hub\intel-isl_MiDaS_master
c:\Users\arnau\anaconda3\envs\pdm\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Loading weights:  None


Using cache found in C:\Users\arnau/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master


In [18]:
input = torch.randn(1, 3, 224, 224)
output = model(input)

In [19]:
output.keys()
print("predicted_depth", output["predicted_depth"].shape)
print("hidden_states", [hs.shape for hs in output["hidden_states"]])

predicted_depth torch.Size([1, 224, 224])
hidden_states [torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024]), torch.Size([1, 197, 1024])]
